<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Aula** | Análise Exploratória de Dados com Python
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)<br>
18/11/2021

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Análise Exploratória;</li>
  <li>Agrupamento;</li>
  <li>Predição.</li>
</ol>

---

# **Aulas**

## 1\. Introdução

### **1.1. Motivação** 

> Um shopping quer **segmentar** seus clientes para melhorar ações de marketing. Para isso, organiza um **programa de fidelidade** no qual os clientes cadastrados trocam suas **notas fiscais** de compra em lojas do shopping por **bilhetes** de um sorteio.

O objetivo é:

 - Encontrar um conjunto de **perfis médios**;
 - **Alocar** os clientes ao perfil mais **próximo**.

Exemplo:

 - Grupos

| Grupo | Idade | Renda |
| ----- | ----- | ----- |
| 1     | 30    | 40    |
| 2     | 57    | 90    |

 - Clientes

| Cliente | Idade | Renda | Grupo |
| ------- | ----- | ----- | ----- |
| 1       | 27    | 50    | 1

Para isso, precisamos responder:

> Qual a **quantidade** de grupos?

> Qual são as **características** de cada grupos?

> Como calcular a **distância** entre o grupo e os clientes?

## 2\. Análise Exploratória

Vamos começar importanto todas as bibliotecas para realizar a análise.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### **2.1. Coleta** 

Neste aula, vamos utilizar dados sobre a **sexo**, **idade**, **renda** e **potencial de compra** de clientes participantes. O conjunto de dados foi inspirado no conjunto de dados do Kaggle, presente neste [link](https://www.kaggle.com/vjchoudhary7/customer-segmentation-tutorial-in-python). 

In [ ]:
!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/mall_customers.csv" -O mall.csv 

### **2.2. Pré-processamento** 

Vamos conhecer um pouco melhor o conjunto de dados.

In [ ]:
mall = pd.read_csv('./mall.csv', sep=',')

In [ ]:
mall.tail(10)

In [ ]:
mall.shape

In [ ]:
mall.info()

Vamos processar os dados para facilitar a sua exploração.

 - Remover a coluna `id`:

In [ ]:
mall = mall.drop('id', axis=1)

In [ ]:
mall.head()

 - Normalizar a coluna `gender`:

In [ ]:
mall['gender'] = mall['gender'].apply(lambda gender: gender.lower())

In [ ]:
mall.head()

### **2.3. Exploração** 

Vamos começar nossa análise resumindo quantitativamente nossos dados.

 - Variáveis numéricas

In [ ]:
mall.describe().T

 - Variáveis categóricas

In [ ]:
mall[['gender']].describe().T

Vamos agora para um resumo qualitativo através de visualizações de dados.

O gráfico de **boxplot** é uma das melhores formas de resumir visualmente um conjunto de dados.

In [ ]:
with sns.axes_style('whitegrid'):

  grafico = sns.boxplot(data=mall, x='age', y="gender", palette="pastel")
  grafico.set(title='Distribuição de Idade por Sexo', xlabel='Idade (anos)',  ylabel='Sexo');

In [ ]:
with sns.axes_style('whitegrid'):

  grafico = sns.boxplot(data=mall, x='income', y="gender", palette="pastel")
  grafico.set(title='Distribuição de Renda por Sexo', xlabel='Renda Anual (milhares de USD)',  ylabel='Sexo');

In [ ]:
with sns.axes_style('whitegrid'):

  grafico = sns.boxplot(data=mall, x='score', y="gender", palette="pastel")
  grafico.set(title='Distribuição de Pontuação por Sexo', xlabel='Pontos',  ylabel='Sexo');

O **pairplot** é um método do pacote Python Seaborn que gera visualizações para cada par de colunas do conjunto de dados.

In [ ]:
with sns.axes_style('whitegrid'):
  grafico = sns.pairplot(data=mall, hue="gender", palette="pastel")

### **2.3. Processamento** 

Nota-se que o perfil de compras de clientes de **sexo** diferentes parece ser o mesmo, não contribuindo assim para a segregação da base. Vamos então selecionar os atributos que de fato podem ajudar na segregação dos cliente: **idade**, **renda** e a **pontuação de compra**.

In [ ]:
data = mall[["age",	"income",	"score"]]

In [ ]:
data.head()

## 3\. Agrupamento

Para agrupar os dados em diferentes grupos, vamos utilizar um modelo de aprendizado de máquina não supervisionado conhecido como **K-Médias**.

### **3.1. Algoritmo** 

Segue uma breve explicação intuitiva sobre o funcionamento do algoritmo do modelo.

In [ ]:
with sns.axes_style('whitegrid'):

  grafico = sns.scatterplot(data=data, x='income', y="score", palette="pastel")
  grafico.set(title='Renda e Pontuação', xlabel='Renda Anual (milhares de USD)',  ylabel='Pontos')
  grafico.figure.set_size_inches(15, 7.5)

### **3.2. Modelagem**

Começamos importando a biblioteca do modelo.

In [ ]:
from sklearn.cluster import KMeans

Vamos treinar o modelo com 5 grupos, também conhecidos como **clusters**.

In [ ]:
model = KMeans(n_clusters=5)
model = model.fit(data)

Com o modelo treinado, podemos visualizar os agrupamentos:

In [ ]:
clusters = model.labels_
clusters = pd.DataFrame(clusters, columns=['cluster'])
clustered_data = pd.concat([data, clusters], axis=1)
clustered_data.head()

In [ ]:
with sns.axes_style('whitegrid'):

  grafico = sns.pairplot(data=clustered_data, hue='cluster', palette="pastel")

Pode-se também observar a características dos centróides ou **perfis médios**:

In [ ]:
clusters_centers = model.cluster_centers_
clusters_centers = pd.DataFrame(clusters_centers, columns=['age', 'income', 'score'])
clusters_centers.head()

In [ ]:
with sns.axes_style('whitegrid'):

  fig, ax = plt.subplots()
  sns.scatterplot(data=clustered_data,   x='income', y='score', hue='cluster', palette="pastel", ax=ax)
  sns.scatterplot(data=clusters_centers, x='income', y='score', color='black', ax=ax)

### **3.3. Avaliação** 

> Seria 5 a melhor quantidade de grupos?

Para responder essa pergunta, utilizamos o **método do cotovelo** que busca balancear erro e quantidade de grupos.

In [ ]:
wcss = []

for k in range(1, 11):

  model = KMeans(n_clusters=k)
  model = model.fit(data)
  wcss.append(model.inertia_)

In [ ]:
with sns.axes_style('whitegrid'):

  grafico = sns.lineplot(x=range(1, 11), y=wcss, marker="8", palette="pastel")
  grafico.set(title='Método do Cotovelo', ylabel='WCSS (Erro)', xlabel='Qtd. clusters');

De acordo com o método do cotovelo, vamos seguir com 5 grupos.

In [ ]:
model = KMeans(n_clusters=5)
model = model.fit(data)

## 4\. Predição

Com o modelo treinado, avaliado e selecionado, podemos utiliza-lo para resolver os problemas reais que motivaram sua construção.

### **4.1. Estudo dos clusters** 

Para direcionar as ações de marketing, precisamos entender melhor o perfil dos clientes dos clusters de interesse através de técnicas de analise exploratória de dados (EDA). Exemplo:

 - **Cluster ?**

São cliente com **alta renda** mas **baixo potencial** de compras médio. Poderiam estar gastando mais.

In [ ]:
with sns.axes_style('whitegrid'):

  grafico = sns.scatterplot(data=clustered_data.query('cluster == 1'), x="age", y="income", hue="cluster", palette="pastel")
  grafico.set(title='Cluster 1 | Distribuição de Renda e Idade', xlabel='Idade (anos)', ylabel='Renda anual (milhares de USD)');
  grafico.get_legend().set_title("Cluster");

**Conclusão:** Focar esforços para um público acima de 35 anos.

### **4.2. Inferência** 

In [ ]:
clusters_centers.head()

 - **Exemplo**: Um cliente com 19 anos, 15.000 USD de renda anual e potencial de compra de 39:


In [ ]:
cliente = pd.DataFrame.from_records(dict(age=[19], income=[15], score=[39]))

In [ ]:
cliente.head()

In [ ]:
int(model.predict(cliente))

Conclui-se então que o cliente pertence ao cluster **?** de baixa renda, idade e potencial de compra.